In [ ]:
import psycopg2
import sys, nltk, re, string, heapq, gensim

from psycopg2 import Error
import math
from matplotlib import pyplot as plt
import pandas as pd
import pandas.io.sql as sqlio

import scikitplot as skplt
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
import numpy as np

def crearConexion():
    
    connection = psycopg2.connect(user = "m.soto.montesinos",
                                      password = "1234",
                                      host = "127.0.0.1",
                                      port = "5432",
                                      database = "twitterBBVA")

    return connection

def obtenerTweets(tabla):

    try:
        
        connection = crearConexion()
        query = """
                SELECT clasificacion, sentimiento, caso_villarejo, futbol, mayusculas,
                lista_palabras_positivas, lista_palabras_negativas, emoticonos_buenos,
                emoticonos_malos FROM %s;
                """ % (tabla)
        
        tweets = sqlio.read_sql_query(query, connection)
        
    except (Exception, psycopg2.DatabaseError) as error :
    
        print ("Error obteniendo los datos", error)
    
    finally:
        
        if (connection):
            
            connection.close()
            print("PostgreSQL connection is closed")

    return tweets


def main():
    
    training_tweets = obtenerTweets("training_set")
    testing_tweets = obtenerTweets("testing_set")
    
    x_training_tweets = training_tweets.drop(columns = ['clasificacion'])
    y_training_tweets = training_tweets['clasificacion']
    x_testing_tweets = testing_tweets.drop(columns = ['clasificacion'])
    y_testing_tweets = testing_tweets['clasificacion']
    
    colsample_bytree = np.linspace(0.5, 1.0, num=5)
    max_depth = [int(x) for x in np.linspace(start = 3, stop = 10, num = 8)]
    gamma = [int(x) for x in np.linspace(start = 0, stop = 10, num = 11)]
    n_estimators = [int(x) for x in np.linspace(start = 0, stop = 10000, num = 101)]
        
    random_grid = {'hidden_layer_sizes': [(50,50,50), (50,100,50), (100,)],
                   'activation': ['tanh', 'relu'],
                   'solver': ['sgd', 'adam'],
                   'alpha': [0.0001, 0.05],
                   'learning_rate': ['constant','adaptive']}
                   
    #modelo_mlp = MLPClassifier(max_iter = 100000)
    
    parameters = {'solver': ['lbfgs'], 'max_iter': [1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000 ], 'alpha': 10.0 ** -np.arange(1, 10), 'hidden_layer_sizes':np.arange(10, 15), 'random_state':[0,1,2,3,4,5,6,7,8,9]}
    clf = GridSearchCV(MLPClassifier(), parameters, n_jobs=-1)
    clf.fit(x_training_tweets, y_training_tweets)
    print(clf.score(x_training_tweets, y_training_tweets))
    print(clf.best_params_)
        
    #Buscamos todas las combinaciones posibles
    #rf_random = RandomizedSearchCV(estimator = modelo_mlp, 
    #                               param_distributions = random_grid, 
    #                               n_iter = 20)
    
    #Entrenamos el modelo con las posibles combinaciones.
    #rf_random.fit(x_training_tweets, y_training_tweets)
    
    #Obtenemos los parametros con mejor resultado
    #print(rf_random.best_params_)
    #{'solver': 'adam', 'learning_rate': 'constant', 'hidden_layer_sizes': (50, 100, 50),
    #'alpha': 0.05, 'activation': 'relu'}
    
    #modelo_mlp = MLPClassifier(max_iter = 1000000,
    #                           solver= 'adam',
    #                           hidden_layer_sizes = (50, 100, 50),
    #                           alpha = 0.05,
    #                           activation = 'relu')
    #
    #modelo_mlp.fit(x_training_tweets, y_training_tweets)
    
    #score = modelo_mlp.score(x_testing_tweets, y_testing_tweets)
    #predicciones = modelo_mlp.predict(x_testing_tweets)
    
    print('\n\n')
    print("Redes Neuronales")
    print('\n\n')
    #print("Score :" + str(score))
    print('\n\n')
    
    #skplt.metrics.plot_confusion_matrix(y_testing_tweets,
    #                                    predicciones,
    #                                    title = "Matriz Confusión Redes Neuronales",
    #                                    normalize=False)
    #plt.show()

    
if __name__ == '__main__':
    main()




PostgreSQL connection is closed
PostgreSQL connection is closed
